In [ ]:
from run_dialmonkey import main
from collections import namedtuple
# just technical hacks, no need to study this
ARGS = namedtuple('ARGS', ['conf', 'logging_level', 'user_stream_type', 'input_file', 'output_stream_type', 'output_file', 'num_dials'])
ARGS.__new__.__defaults__ = (None,) * len(ARGS._fields)

In [ ]:
# basic dummy configuration
args = ARGS(conf='./conf/sample_conf.yaml', num_dials=1)
main(args)

In [ ]:
# file input, file output configuration
args = ARGS(conf='./conf/public_transport_cs.yaml', num_dials=1)
main(args)

In [ ]:
import os
import logzero
import logging

from dialmonkey.utils import load_conf, run_for_n_iterations, DialMonkeyFormatter
from dialmonkey.conversation_handler import ConversationHandler

# important are the last two lines
args = ARGS(conf='./conf/sample_conf.yaml', num_dials=1, logging_level='ERROR')
# load configuration
conf = load_conf(args.conf)

# setup logging
if args.logging_level:
    conf['logging_level'] = args.logging_level
elif 'logging_level' not in conf:
    conf['logging_level'] = 'NOTSET'

formatter = DialMonkeyFormatter(
    path_prefix=os.path.join(os.path.abspath(os.path.curdir), 'dialmonkey/'),
    # https://docs.python.org/3/library/logging.html#logrecord-attributes
    fmt='%(color)s%(asctime)s [%(levelname)1.1s %(relpath)s:%(lineno)s]%(end_color)s %(message)s',
    datefmt='%H:%M:%S')

logger = logzero.setup_logger(level=getattr(logging, conf['logging_level']),
                              formatter=formatter)

# setup input & output streams
if args.user_stream_type:
    conf['user_stream_type'] = args.user_stream_type
if args.input_file:
    conf['input_file'] = args.input_file
if args.output_stream_type:
    conf['output_stream_type'] = args.output_stream_type
if args.output_file:
    conf['output_file'] = args.output_file

# show the loaded components
handler = ConversationHandler(conf, logger, should_continue=run_for_n_iterations(args.num_dials))
handler.components

In [ ]:
from dialmonkey.dialogue import Dialogue
# show how works one dialogue turn
dial = Dialogue()
system_response = ''
handler._reset_components()
handler._init_components(dial)
user_utterance = handler.user_stream(system_response)
system_response, eod = handler.get_response(dial, user_utterance)
handler.output_stream(system_response)

In [ ]:
dial = Dialogue()
system_response = ''
handler._reset_components()
handler._init_components(dial)
user_utterance = 'hello'
dial.set_user_input(user_utterance)
# run the dialogue pipeline (all components from the config)
for component in handler.components:
    # components pass the same dialogue object
    # the dialogue object is modified by each component
    # one of the components should call set_system_response() and end_dialogue()
    dial = component(dial, handler.logger)
    print(f'after component: {component}')
    print(f'dialogue nlu: {dial.nlu}')
    print(f'dialogue state: {dial.state}')
    print(f'system_response: "{dial.system}"')
    print('-'*80)

print(f"dial.state == dial['state']: {dial.state == dial['state']}")

In [ ]:
# da items are structured
for act_item in dial.nlu:
    print(act_item)
    print(f'intent={act_item.intent}, slot={act_item.slot}, value={act_item.value}, confidence={act_item.confidence}')

In [ ]:
from dialmonkey.da import DAI,DA
# DAI can be constructed explicitly...
da_item = DAI(intent='pay',slot='object',value='taxes', confidence=.5)
da = DA([da_item])
for act in da: # da.dais is iterable
    print(f'DA 1 act: {act}')

# ... or implicitly
da_item2 = DAI.parse('pay(object=taxes)/0.5')
da2 = DA([da_item2])
for act in da2:
    print(f'DA 2 act: {act}')

# DA can be cnstructed straight from str representation...
da3 = DA.parse('pay(object=taxes)/0.5')

print(f'da == da2 == da3, {da == da2 == da3}')

# ... even with multiple items
da4 = DA.parse('pay(object=taxes)/0.500&pay(object=Netflix)/0.999')
for act in da4: 
    print(f'DA 4 act: {act}')
print(f'da == da4, {da == da4}')